In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf  # for data preprocessing
from tensorflow import keras
import nibabel as nib

from scipy import ndimage
from sklearn.preprocessing import StandardScaler
import random
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling2D,Flatten,concatenate,BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import regularizers
from collections import Counter
import sys

# Reading and ploting single nii file

In [ ]:
root_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/test/control/spect/"
file_path = root_path + "3104/preprocessed/PPMI_3104_NM_Reconstructed_DaTSCAN_Br_20121011134355542_1_S117556_spect_resampled_registered.nii.gz"
my_nifti = nib.load(file_path).get_fdata()

# get the shape of your NIfTI
print(my_nifti.shape)

# Rotate the image data 90 degrees counterclockwise
rotated_img_data = np.rot90(my_nifti)

# Choose the slice index you want to save
slice_index = 78  # Change this to the index of the slice you want to save

# Get the selected slice
selected_slice = rotated_img_data[:, :, slice_index]

# Repeat the slice along a new axis to create an RGB-like dimension
rgb_slice = np.repeat(selected_slice[:, :, np.newaxis], 3, axis=2)

print(f"min value: {rgb_slice.min()}, max value: {rgb_slice.max()}")
print(f"dimensions: {rgb_slice.shape}")

# # access it as 3D numpy array
# nifti_slice = my_nifti[:,:,30]

# # Rotate the slice 90 degrees counterclockwise
# rotated_img_data = np.rot90(nifti_slice)

# display the slice
plt.imshow(selected_slice, cmap="gray")
plt.show()

In [ ]:
# Save the selected slice as a NumPy file
np.save('rgb_slice.npy', rgb_slice)

In [ ]:
#reading and plotting the np slice saved
np_file = "/home/Data/franklin/Doctorado/parkinson/projects/parcellation_translation/notebooks/rgb_slice.npy"
np_image = np.load(np_file)
print(f"min value: {np_image.min()}, max value: {np_image.max()}")
print(np_image.shape)
# Normalize the RGB data to the range [0, 1]
rgb_slice_normalized = np_image / np.max(np_image)
# Plot the array data
plt.imshow(rgb_slice_normalized, cmap='gray')
plt.show()

### Saving npy files in a similar structure like the PNG ones. 
Filterin the slices regarding the following criteria:
* Parkinson and control populations:

	* SPECT: * CONTROL---->[34, 47]
		 * PD---->[29, 49]

	* MRI:	 * CONTROL---->[55, 68]
    	 * PD---->[55, 68]

* SWEDD populations
	* SPECT: [32, 50]
	* MRI: [51, 65]

* PRODROMAL population------> **it needs to be revised**
	* MRI: []


In [ ]:
def preprocess_nii(file_path, low_idx, top_idx, save_path, case_name, modal_name):
    
    # get the nii data
    my_nifti = nib.load(file_path).get_fdata()
    # Rotate the nii data 90 degrees counterclockwise
    rotated_img_data = np.rot90(my_nifti)  
    # Choose the slice index you want to save
    for slice_index in range(low_idx, top_idx+1, 1):
        
        # Get the selected slice
        selected_slice = rotated_img_data[:, :, slice_index]

        # Repeat the slice along a new axis to create an RGB-like dimension
        rgb_slice = np.repeat(selected_slice[:, :, np.newaxis], 3, axis=2)
        
        # Save the selected slice as a NumPy file
        directory = save_path + case_name + "/"
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        np_file_name = directory + case_name + "_" + modal_name + "_slice_" + str(slice_index) + ".npy"
        #print(f"saving {file_path} on: {np_file_name}")
        np.save(np_file_name, rgb_slice)
        

**For Parkinson and Control populations**

In [ ]:
root_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered"
split = "test"
group = "parkinson"
modality = "spect"
source_path = root_path + "/" + split + "/" + group + "/" + modality
cases = sorted(os.listdir(source_path))

for case in cases:
    print(case)
    preprocessed_path = source_path + "/" + case + "/preprocessed/"
    files = sorted(os.listdir(preprocessed_path))
    
    # preprocessed file identification and slices selection
    if modality == "spect":
        nii_file = [file for file in files if file.endswith("_resampled.nii.gz")][0]
        case_name = nii_file.split("_")[1]
        modal_name = nii_file.split("_")[4]
        nii_path = preprocessed_path + nii_file
        
        if group == "control":
            low_slice = 34
            top_slice = 47
        else:
            low_slice = 29
            top_slice = 49
    else:
        nii_file = [file for file in files if file.endswith("rigid_registered.nii.gz")][0]
        case_name = nii_file.split("_")[1]
        modal_name = nii_file.split("_")[3]
        nii_path = preprocessed_path + nii_file
        low_slice = 55
        top_slice = 68
        
    # data preprocessing: nii rotatation, slice selection and numpy save
    save_path = root_path + "/" + split + "/" + group + "/" + "extension" + "/npyFiles/" + modality + "/"
    preprocess_nii(nii_path, low_slice, top_slice, save_path, case_name, modal_name)  
    
         

**For SWEDD populations**

In [ ]:
root_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered"
group = "swedd"
modality = "spect"
source_path = root_path + "/" + group + "/" + modality#---->
cases = sorted(os.listdir(source_path))

for case in cases:
    print(case)
    preprocessed_path = source_path + "/" + case + "/preprocessed/"
    files = sorted(os.listdir(preprocessed_path))
    
    # preprocessed file identification and slices selection
    if modality == "spect":
        nii_file = [file for file in files if file.endswith("_registered.nii.gz")][0]
        case_name = nii_file.split("_")[0]
        modal_name = nii_file.split("_")[5]
        nii_path = preprocessed_path + nii_file
        low_slice = 32
        top_slice = 50
               
    else:
        nii_file = [file for file in files if file.endswith("rigid_registered.nii.gz")][0]
        case_name = nii_file.split("_")[1]
        modal_name = nii_file.split("_")[3]
        nii_path = preprocessed_path + nii_file
        low_slice = 59
        top_slice = 108
       
     
    # data preprocessing: nii rotatation, slice selection and numpy save
    save_path = root_path + "/" + group + "/" + "extension" + "/npyFiles/" + modality + "/"
    preprocess_nii(nii_path, low_slice, top_slice, save_path, case_name, modal_name) 

# CNN training

### Helper functions

In [ ]:
def read_np_file(filepath):
    """Read and load volume"""
    # Read file
    scan = np.load(filepath)
    #new lines
    if scan.shape != (218, 182, 3):
        scan = np.resize(scan, (218, 182, 3))
    else:
        None
    
    return scan

def normalize(slice, path):
    # Scale the data between 0 and 1
    min_val = np.min(slice)
    max_val = np.max(slice)
    denominador = (max_val - min_val)
    scaled_data = (slice - min_val) / denominador
    
    if denominador <=0:
        print("ERROR")
        print("min_val: ", min_val)
        print("max_val: ", max_val)
        print("path: ", path)
    
    # Apply Z normalization 
    mean = np.mean(scaled_data)
    std_dev = np.std(scaled_data)
    normalized_data = (scaled_data - mean) / std_dev
    return normalized_data

def process_scan(path):
    """Read and resize volume"""
    # Read scan
    slice = read_np_file(path)
    # Normalize
    slice = normalize(slice, path)
    return slice

def load_npy_paths(split, group, modality):
    """Read each nii path regardless of the split and the group""" 
    root_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/"
    cases = sorted(os.listdir(root_path + split + "/" + group + "/extension/npyFiles/" + modality))
    paths = []
    for case in cases:
        case_path = root_path + split + "/" + group + "/extension/npyFiles/" + modality + "/" + case + "/"
        files = os.listdir(case_path)
        
        for file in files:
            file_path = case_path + file
            paths.append(file_path)
    return paths    

## Data augmentation

In [ ]:
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        # volume[volume < 0] = 0
        # volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    #volume = rotate(volume)
    #volume = tf.expand_dims(volume, axis=3)
    return volume, label


def test_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

**Getting the nii path for each split**

In [ ]:
root_path = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/"
##===== for control group
#train
split = "train"
group = "control"
modality = "spect"
control_train_paths = load_npy_paths(split, group, modality)
#test
split = "test"
control_test_paths = load_npy_paths(split, group, modality)

##===== for parkinson group
#train
split = "train"
group = "parkinson"
modality = "spect"
parkinson_train_paths = load_npy_paths(split, group, modality)
#test
split = "test"
parkinson_test_paths = load_npy_paths(split, group, modality)

########## labels ####################
# assign 1 for parkinson and 0 for the control ones.
control_train_labels = np.array([0 for _ in range(len(control_train_paths))])
control_test_labels = np.array([0 for _ in range(len(control_test_paths))])
parkinson_train_labels = np.array([1 for _ in range(len(parkinson_train_paths))])
parkinson_test_labels = np.array([1 for _ in range(len(parkinson_test_paths))])

## in summay:
print("========== paths info ================")
print("Control train paths: ", len(control_train_paths))
print("Control test paths: ", len(control_test_paths))
print("Parkinson train paths: ", len(parkinson_train_paths))
print("Parkinson test paths: ", len(parkinson_test_paths))

print("========== labels info ================")

print("Control train labels: ", len(control_train_labels))
print("Control test labels: ", len(control_test_labels))
print("Parkinson train labels: ", len(parkinson_train_labels))
print("Parkinson test labels: ", len(parkinson_test_labels))


## **Building the train and test datagenerators**

In [ ]:
def data_generator(file_paths, labels, batch_size):
    num_samples = len(file_paths)
    while True:
        # Shuffle the data for each epoch
        indices = np.random.permutation(num_samples)
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_indices = indices[start_idx:end_idx]
            batch_files = [file_paths[i] for i in batch_indices]
            batch_labels = labels[batch_indices]
            
            # Load and preprocess the batch of data
            X_batch = []
            for file_path in batch_files:
                # Preprocess data as needed
                data = process_scan(file_path)
                X_batch.append(data)
            X_batch = np.array(X_batch)
            
            # Convert NumPy arrays to TensorFlow tensors
            X_batch_tf = tf.convert_to_tensor(X_batch)
            y_batch_tf = tf.convert_to_tensor(batch_labels)
            
            yield X_batch_tf, y_batch_tf

In [ ]:
batch_size = 8

X_train_files = np.concatenate((control_train_paths, parkinson_train_paths), axis=0)
y_train = np.concatenate((control_train_labels, parkinson_train_labels), axis=0)

X_test_files = np.concatenate((control_test_paths, parkinson_test_paths), axis=0)
y_test = np.concatenate((control_test_labels, parkinson_test_labels), axis=0)

# Create generators for training and validation data
train_generator = data_generator(X_train_files, y_train, batch_size)
val_generator = data_generator(X_test_files, y_test, batch_size)

# **Defining the 2D CNN**

In [ ]:
input_shape = (218, 182, 3)
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        
#making the transfer learning
for layer in base_model.layers[:10]:
    layer.trainable = False
for layer in base_model.layers[10:]:
    layer.trainable = True 
    
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten(name='flatten')(x)
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc1')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc2')(x)
preds = tf.keras.layers.Dense(2, activation='softmax', name='predictions')(x)
custom_model = Model(inputs=base_model.input, outputs=preds)

print(custom_model.summary())

In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000, 
                                                             decay_rate=0.96, staircase=True)

custom_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                     optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=["acc"], run_eagerly=True)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.keras", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=5)

# Train the model, doing validation at the end of each epoch
num_epochs = 100
# Calculate the number of batches per epoch
num_train_samples = len(X_train_files)
num_valid_samples = len(X_test_files)

num_train_steps = num_train_samples // batch_size
num_valid_steps = num_valid_samples // batch_size
#save model
save_path = "/home/Data/franklin/Doctorado/parkinson/projects/parcellation_translation/models/embc_extension/classifier/mri_spect/nii_files/"

# Redirect stdout to suppress output
class SuppressOutput:
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._stdout
ref_val_acc = 0
ref_val_loss = 1
for epoch in range(num_epochs):
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Training loop
    train_loss = 0.0
    train_accuracy = 0.0
    for _ in range(num_train_steps):
        X_train_batch, y_train_batch = next(train_generator)
        with SuppressOutput():
            train_metrics = custom_model.train_on_batch(X_train_batch, y_train_batch)
        train_loss += train_metrics[0]
        train_accuracy += train_metrics[1]
    
    train_loss /= num_train_steps
    train_accuracy /= num_train_steps
    print(f"Training Loss: {train_loss}, Training Accuracy: {train_accuracy}")
    
    # Validation loop
    valid_loss = 0.0
    valid_accuracy = 0.0
    for _ in range(num_valid_steps):
        X_valid_batch, y_valid_batch = next(val_generator)
        valid_metrics = custom_model.evaluate(X_valid_batch, y_valid_batch, verbose=0)
        valid_loss += valid_metrics[0]
        valid_accuracy += valid_metrics[1]
    
    valid_loss /= num_valid_steps
    valid_accuracy /= num_valid_steps
    print(f"Validation Loss: {valid_loss}, Validation Accuracy: {valid_accuracy}")
    if valid_loss < ref_val_loss:
        ref_val_loss = valid_accuracy
        #saving the model
        model_name = "vgg16LossReference.h5"
        custom_model.save(save_path+model_name)
    if valid_accuracy > ref_val_acc:
        ref_val_acc = valid_accuracy
        #saving the model
        model_name = "vgg16AccReference.h5"
        custom_model.save(save_path+model_name)
        